# Serve Docs
This is a utlity to serve the docs locally.  It provides a web UI with links to the raw json and document viewer for each document.

1. Optional - configure the port number. 
2. Sometimes necessary - restart your kernel
3. Run the cell
4. A list of links to to the documents as raw json and in document viewer will be visible at this url http://localhost:8083/


In [1]:
import os
import http.server
import time

# configure local port to serve document json files
port_number = 8083
port_number_string = str(port_number)

# server class
class CustomHTTPRequestHandler(http.server.SimpleHTTPRequestHandler):
    def end_headers(self):
        self.send_header('Access-Control-Allow-Origin', '*')
        self.send_header('Access-Control-Allow-Methods', 'GET')
        self.send_header('Access-Control-Allow-Headers', 'Content-Type')
        if self.path.endswith(".json"):
            self.send_header("Content-Type", 'application/json')
        else:
            self.send_header("Content-Type", 'text/html')
        http.server.SimpleHTTPRequestHandler.end_headers(self)

    def do_GET(self):
        if self.path == '/':
            self.path = '/index.html'
        return http.server.SimpleHTTPRequestHandler.do_GET(self)

# make sure we are in the right place
starting_dir = os.getcwd()
if starting_dir.endswith('documents'):
    os.chdir('..')

current_dir = os.getcwd()
if not current_dir.endswith('docsfiles'):
    print('This script must be run from the `docsfiles` directory. Quit and restart from the correct directory.')
    exit()

documents_dir = "./documents/"
os.chdir(documents_dir)

# write an html file to serve
with open('index.html', 'w') as f:
    style_open = "<style>"
    table_style = "table {font-size: 12px; font-family: sans-serif;}"
    table_cell_style = "td {text-align: left; padding: 4px;}"
    style_close = "</style>"
    style = style_open + table_style + table_cell_style + style_close
    html_top = "<html><body>"
    heading = "<h1>Documents:</h1>"
    table_start = "<table><thead><tr><th>Document</th><th>JSON Link</th><th>Document View</th></tr></thead><tbody>"
    html_start = style + html_top + heading + table_start
    f.write(html_start)

    for document in sorted(os.listdir('.')):
        base_url = "https://collaborative-learning.concord.org"
        uri = "/branch/master/doc-editor.html"
        paramStr = "?unit=example&document="
        service_url = "http://localhost:" + port_number_string
        # e.g. https://collaborative-learning.concord.org/branch/master/doc-editor.html?unit=example&document=http://localhost:8083/document3.json
        timestamp = str(time.time())
        doc_link = base_url + uri + paramStr + service_url + "/" + document
        title_cell = f'<tr><td>{document}</td>'
        json_cell = f'<td><a target="_blank" href="{document}">json</a></td>'
        doc_cell = f'<td><a target="_blank" href="{doc_link}">doc</a></td></tr>'
        f.write(title_cell + json_cell + doc_cell)

    table_end = "</tbody></table>"
    html_bottom = "</body></html>"
    html_end = table_end + html_bottom
    f.write(html_end)

handler = CustomHTTPRequestHandler
httpd = http.server.HTTPServer(('localhost', port_number), handler)

print(f"Serving HTTP on localhost port {port_number_string}...")
print(f"open {service_url} in your browser")

httpd.serve_forever()



Serving HTTP on localhost port 8083...
open http://localhost:8083 in your browser


127.0.0.1 - - [15/Nov/2023 08:23:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2023 08:24:49] "GET /document100.json HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2023 08:24:53] code 501, message Unsupported method ('OPTIONS')
127.0.0.1 - - [15/Nov/2023 08:24:53] "OPTIONS /document118.json HTTP/1.1" 501 -
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 52531)
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Library/Fr